In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
source('0_parameters.sh')


In [3]:
master_phe_info %>%
fread(colClasses = c('#GBE_ID'='character')) %>%
rename('GBE_ID'='#GBE_ID') -> master_phe_info_df


In [4]:
master_phe_info_df %>% drop_na(FIELD) %>% group_by(FIELD) %>%
summarise(n_GBE_IDs = n(), n_uniq_Ns = length(unique(N))) %>%
filter(n_GBE_IDs != n_uniq_Ns) -> potential_duplicates_df


In [5]:
potential_duplicates_df %>% dim()

[1] 16  3

In [6]:
potential_duplicates_df %>%
rename('#FIELD' = 'FIELD') %>%
fwrite(potential_dups_UKB_f, sep='\t', na = "NA", quote=F)


In [8]:
master_phe_info_df %>% filter(FIELD %in% (potential_duplicates_df %>% pull(FIELD))) %>%
arrange(FIELD, N) %>%
select(GBE_ID, FIELD, GBE_NAME, N) %>%
left_join(potential_duplicates_df, by='FIELD') %>%
mutate(AMS_URL=paste0('https://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=', FIELD)) %>%
rename('#GBE_ID' = 'GBE_ID') %>%
fwrite(potential_dups_GBE_f, sep='\t', na = "NA", quote=F)
